In [2]:
# !pip install langchain
# !pip install sentence_transformers
# !pip install pypdf2
# !pip install transformers
# !pip install sentence-transformers
# !pip install regex
# !pip install langchain-community
# !pip install InstructorEmbedding
# !pip install huggingface-hub
# !pip install faiss-cpu
# !pip install python-dotenv


  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.27-py3-none-any.whl.metadata (8.2 kB)
  Using cached langchain_community-0.0.26-py3-none-any.whl.metadata (8.2 kB)
  Using cached langchain_community-0.0.25-py3-none-any.whl.metadata (8.1 kB)
  Using cached langchain_community-0.0.24-py3-none-any.whl.metadata (8.1 kB)
  Using cached langchain_community-0.0.23-py3-none-any.whl.metadata (8.1 kB)
  Using cached langchain_community-0.0.22-py3-none-any.whl.metadata (8.1 kB)
  Using cached langchain_community-0.0.21-py3-none-any.whl.metadata (8.1 kB)
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.20-py3-none-any.whl.metadata (8.1 kB)

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python312\\scripts\\langsmith.exe'
Consider using the `--user` option or check the permissions.



  Using cached langsmith-0.1.27-py3-none-any.whl.metadata (13 kB)
Using cached langsmith-0.1.27-py3-none-any.whl (68 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python312\\Scripts\\langsmith.exe' -> 'C:\\Python312\\Scripts\\langsmith.exe.deleteme'
Consider using the `--user` option or check the permissions.



In [3]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
import re

In [4]:
# to get pdf texts
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)

        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [14]:

# preprocess the text

def preprocess(raw_text):

    # Remove extra whitespaces and newline characters
    cleaned_text = re.sub(r'\s+', ' ', raw_text)
    # Remove leading and trailing whitespaces
    # cleaned_text = cleaned_text.strip()
    return cleaned_text

In [15]:
# to get text chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1500,
        chunk_overlap=100,
        length_function=len
    )

    chunks = text_splitter.split_text(text)
    return chunks

In [16]:
# make a vector store if you want through api
def get_vectorstore(text_chunks):
    inference_api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")
    embeddings = HuggingFaceInferenceAPIEmbeddings(api_key = inference_api_key, model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts = text_chunks, embedding = embeddings)
    return vectorstore

In [17]:
# # without using api
# from langchain_community.embeddings import HuggingFaceEmbeddings

# def get_vectorstore(text_chunks):
    
#     embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")
#     vectorstore = FAISS.from_texts(text_chunks, embeddings)
#     return vectorstore

In [42]:
# get the conversation chain
def get_conversation_chain(vectorstore):
    
    llm = HuggingFaceHub( repo_id="mistralai/Mistral-7B-Instruct-v0.2", model_kwargs={"temperature": 0.5, "max_length": 64, "max_new_tokens":500})

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type='stuff',
        retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    )
    return conversation_chain

In [43]:
# Define chat function

def conversation_chat(conversation_chain, query):
    result = conversation_chain({"question": query, 'chat_history': []})
    return result["answer"]

In [44]:
# main function

def main(pdf_docs):
    
    # Get PDF text
    raw_text = get_pdf_text(pdf_docs)

    # reprocess text
    text = preprocess(raw_text)

    # Get text chunks
    text_chunks = get_text_chunks(text)
    print(text_chunks)
    # Create vector store
    vectorstore = get_vectorstore(text_chunks)

    # creating the chain
    conversation_chain = get_conversation_chain(vectorstore)

    return conversation_chain


In [47]:
# enter the docs here
pdf_docs = ["./Dataset/halliburton-announces-1st-quarter-2023-results.pdf", "./Dataset/FactSheet_Climate_Jobs_2024.pdf", "./Dataset/halliburton-announces-2nd-quarter-2023-results.pdf", "./Dataset/halliburton-announces-4th-quarter-2023-results"]

conversation_chain = main(pdf_docs)

['FOR IMMEDIATE RELEASE HALLIBURTON ANNOUNCES FIRST QUARTER 2023 RESULTS • Net income of $0.72 per diluted share. • Net income per diluted share more than doubled from Q1 2022. • Revenue of $5.7 billion , increased 33% year-over-year. • Operating margin of 17.2%, increased 530 basis points year -over-year. HOUSTON – April 25, 2023 – Halliburton Company (NYSE: HAL) announced today net income of $651 million , or $0.72 per diluted share, for the first quarter of 2023 . This compares to net income for the first quarter of 2022 of $263 million, or $0.29 per diluted share. Halliburton\'s total revenue for the first quarter of 2023 was $5.7 billion compared to total revenue of $4.3 billion in the first quarter of 2022 . Repo rted operating income was $977 million in the first quarter of 2023 compared to reported operating income of $511 million and adjusted operating income, excluding impairments and other charges, of $533 million in the first quarter of 2022 . “Halliburton’s performance in 

In [48]:
# ask the question here 
user_input = "North America revenue in the fourth quarter of 2023 was"

output = conversation_chat(conversation_chain ,user_input)

In [49]:
# using regular expression to get the helpful answer

import re
match = re.findall(r'\nHelpful Answer: (.*)', output)

for item in match:
    print(item)

To find the North America revenue for the fourth quarter of 2023, you would need to refer to Halliburton's fourth quarter 2023 financial results press release, which has not been provided in the context.
